<a href="https://colab.research.google.com/github/yasserius/video_facial_recognition/blob/main/video_facial_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: [machine learning mastery](https://machinelearningmastery.com/how-to-perform-face-recognition-with-vggface2-convolutional-neural-network-in-keras/)
This tutorial is amazing and most of the code is from there.

#Download the model and face detection libraries

VGGFace returns a facial embedding of dimensions 

In [ ]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-lg8w427h
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-lg8w427h
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp36-none-any.whl size=8311 sha256=4fec351eb4e07df95727e075675dfd903813b9ba6840fa87b39e838bed569e5a
  Stored in directory: /tmp/pip-ephem-wheel-cache-lb90cjf6/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


[MTCNN](https://github.com/ipazc/mtcnn) is a very accurate face detection program, with an easy to use face extractor functions.

In [ ]:
!pip install mtcnn

     |████████████████████████████████| 2.3MB 11.2MB/s 


#Helper functions
Contains the face detection, face embeddings model and embeddings comparison functions needed.

In [ ]:
!pip install keras_applications

     |████████████████████████████████| 51kB 5.7MB/s 


In [ ]:
# source https://machinelearningmastery.com/how-to-perform-face-recognition-with-vggface2-convolutional-neural-network-in-keras/

from matplotlib import pyplot
from PIL import Image
import numpy as np
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
 
def extract_face_from_numpy(pixels, required_size=(224, 224), display=False):
	detector = MTCNN()
	results = detector.detect_faces(pixels)
	num_faces = len(results)
 
	if display:
		print("%d face(s) detected." % num_faces)
	
	faces_array = []
	for i in range(num_faces):
		x1, y1, width, height = results[i]['box']
		x2, y2 = x1 + width, y1 + height
		face = pixels[y1:y2, x1:x2]
		image = Image.fromarray(face)
		image = image.resize(required_size)
		if display:
			pyplot.subplot(1, num_faces, i+1)
			pyplot.imshow(image)
			pyplot.xticks([])
			pyplot.yticks([])
		faces_array.append(asarray(image))
	
	return faces_array

def extract_face_from_file(filename, required_size=(224, 224), display=True):
	pixels = pyplot.imread(filename)
	detector = MTCNN()
	results = detector.detect_faces(pixels)
	num_faces = len(results)
 
	if display:
		print("%d face(s) detected." % num_faces)
	
	faces_array = []
	for i in range(num_faces):
		x1, y1, width, height = results[i]['box']
		x2, y2 = x1 + width, y1 + height
		face = pixels[y1:y2, x1:x2]
		image = Image.fromarray(face)
		image = image.resize(required_size)
		if display:
			pyplot.subplot(1, num_faces, i+1)
			pyplot.imshow(image)
			pyplot.xticks([])
			pyplot.yticks([])
		faces_array.append(asarray(image))
	
	return faces_array



def get_embeddings(face):
	face = asarray(face, 'float32')
	face = np.expand_dims(face, axis=0)
	face = preprocess_input(face, version=2)
	model = VGGFace(model='resnet50',
	                include_top=False,
									input_shape=(224, 224, 3),
									pooling='avg')
	embeddings = model.predict(face)
	embeddings = np.squeeze(embeddings, axis=0)
	
	return embeddings


def is_match(face1, face2, thresh=0.5, display=True):
	embed1 = get_embeddings(face1)
	embed2 = get_embeddings(face2)

	score = cosine(embed1, embed2)
 
	if score <= thresh:
		# print('Match (%.3f <= %.3f)' % (score, thresh))
		return True
	else:
		# print('NOT a match (%.3f > %.3f)' % (score, thresh))
		return False
	
	# if display:
	# 	pyplot.subplot(1, 2, 1)
	# 	pyplot.imshow(face1)
	# 	pyplot.xticks([])
	# 	pyplot.yticks([])

	# 	pyplot.subplot(1, 2, 2)
	# 	pyplot.imshow(face2)
	# 	pyplot.xticks([])
	# 	pyplot.yticks([])

# Run Facial Recognition on Video



In [ ]:
!pip install pytube

In [ ]:
from pytube import YouTube
YouTube('https://www.youtube.com/watch?v=ydo7M_9EGeQ').streams.first().download(filename="video1")

'/content/video1.mp4'

In [ ]:
!pip install av

In [ ]:
def check_face(new_face, unique_faces):
  if len(unique_faces) == 0:
    return True, 0 # id is zero since first face
  
  for id, face in enumerate(unique_faces):
    if is_match(face, new_face):
      return False, id # returns old face id
    else:
      return True, len(unique_faces) # if is length since new face

In [ ]:
import matplotlib.pyplot as plt
import time
import random
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import av

v = av.open('/content/video1.mp4')
t = 0
unique_faces = []
face_timestamps = {}

for packet in v.demux():
    for frame in packet.decode():
      if type(frame) == av.video.frame.VideoFrame:
        img = frame.to_image()  # PIL/Pillow image
        arr = np.asarray(img)  # numpy array
        # plt.imshow(arr)
        # plt.show()
        faces = extract_face_from_numpy(arr)

        if len(faces) > 0:
          face_ids = []
          for face in faces:
            new, id = check_face(face, unique_faces)
            face_ids.append(id)
            print(t, new, id)
            if new:
              unique_faces.append(face)
              print(t, "new face")
            
          # print("Face count", len(all_faces.keys()))
          if len(face_ids) != 0:
            for idx in face_ids:
              if idx not in face_timestamps.keys():
                face_timestamps[idx] = []
              
              face_timestamps[idx].append(t)
              
        t += 1
        print(t, face_timestamps)

1 {}
2 {}
3 {}
4 {}
5 {}
6 {}
7 {}
8 {}
9 {}
10 {}
11 {}
12 {}
13 {}
14 {}
15 {}
16 {}
17 {}
18 {}
19 {}
20 {}
21 {}
22 {}
23 {}
24 {}
25 {}
26 {}
27 {}
28 {}
29 {}
30 {}
31 {}
32 {}
33 {}
34 {}
35 {}
36 {}
37 {}
38 {}
39 {}
40 {}
41 {}
42 {}
43 {}
44 {}
45 {}
46 {}
47 {}
48 {}
49 {}
50 {}
51 {}
52 {}
53 {}
54 {}
55 {}
56 {}
57 {}
58 {}
59 {}
60 {}
61 {}
62 {}
63 {}
64 {}
65 {}
66 {}
67 {}
68 {}
69 {}
70 {}
71 {}
72 {}
73 {}
74 {}
75 {}
76 {}
77 {}
78 {}
79 {}
80 {}
81 {}
82 {}
83 {}
84 {}
85 {}
86 {}
87 {}
88 {}
89 {}
90 {}
91 {}
92 {}
93 {}
94 {}
95 {}
96 {}
97 {}
98 {}
99 {}
100 {}
101 {}
102 {}
103 {}
104 {}
105 {}
106 {}
107 {}
108 {}
109 {}
110 {}
111 {}
112 {}
113 {}
114 {}
115 {}
116 {}
117 {}
118 {}
119 {}
120 {}
121 {}
122 {}
123 {}
124 {}
125 {}
126 {}
127 {}
128 {}
129 {}
130 {}
131 {}
132 {}
133 {}
134 {}
135 {}
136 {}
137 {}
138 {}
139 {}
140 {}
141 {}
142 {}
143 {}
144 {}
145 {}
146 {}
147 {}
148 {}
149 {}
150 {}
151 {}
152 {}
153 {}
154 {}
155 {}
156 {}
157 {}
158 {}
15

ResourceExhaustedError: ignored